In [ ]:
C:\Users\srini\OneDrive\Desktop\webScraping

# Web Scraping Using Selenium
## Importing libraries and packages


In [62]:
# Installing selenium
! pip install selenium


In [63]:
# Importing libraries
from time import sleep, strftime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np

In [64]:
'''
Passing path of the chrome webdriver
'''
chromedriver_path = r'C:\Users\srini\OneDrive\Desktop\webScraping\chromedriver.exe'


In [65]:

'''
steps to reach out to the main page where the table is present 
'''
driver = webdriver.Chrome(executable_path=chromedriver_path)
# driver.maximize_window()
# sleep(10)
# driver.get("https://www.iplt20.com")
# sleep(10)
# driver.find_elements_by_xpath("//div[contains(@class, 'main-nav__drop-down')]")[2].click()
# sleep(2)
# driver.find_elements_by_xpath("//a[contains(@class, 'main-nav__link') and contains(text(), 'By Season')]")[0].click()
# sleep(5)


In [66]:
'''
Since this IPL website doesn't as for any login id or password, we can extract out data from the URL of the 
page where table is present.
'''
season_url = "https://www.iplt20.com/stats/{}/most-runs" #leaving season part as blank that we'd fill in the later part 


In [55]:
del df

In [69]:
'''
defining main function for scraping data. This function will scrape all the texts from the data and store it into df_runs
by appending it on an iterative call
'''
def get_text(webelement):
    return webelement.get_attribute("innerText").replace("\n", " ")

def page_scrape(season):
    '''
    Functions to scrape a page details for a particular year
    '''
    table_rows = driver.find_elements_by_xpath("//table//tr[contains(@class, 'js-row')]")
    for ele in table_rows :
        row_data = list(map(get_text, ele.find_elements_by_tag_name("td")))
        if (row_data[0]=='1'):
            row_data.extend([ele.get_attribute('data-nationality'), season, ele.get_attribute('class').split()[2]])
        else:
            row_data.extend([ele.get_attribute('data-nationality'), season, ele.get_attribute('class').split()[1]])
        df_runs.loc[len(df_runs)] = row_data


In [70]:
driver.get("https://www.iplt20.com/stats/{}/most-runs".format('2021'))
table_heading = driver.find_element_by_xpath("//table//tr[contains(@class, 'top-players__header')]")
globals()['column_values'] = list(table_heading.get_attribute("innerText").split())
column_values.extend(['Nationality', 'Season', 'Team'])

df_runs = pd.DataFrame(columns = column_values)

for season in range(2021, 2007, -1):
    driver.get(season_url.format(season))
    page_scrape(season)


In [71]:
df_runs['status']=np.where(df_runs['HS'].str.contains(r'[*]'),'Not Out','Out')

In [80]:

'''
defining main function for scraping wicket data. This function will scrape all the texts from the wickets page and store it into a dataframe
df_wickets by appending it on an iterative call
'''
season_url= "https://www.iplt20.com/stats/{}/most-wickets"


def get_text(webelement):
    return webelement.get_attribute("innerText").replace("\n", " ")

def page_scrape(season):
    '''
    Functions to scrape a page details for a particular year
    '''
    table_rows = driver.find_elements_by_xpath("//table//tr[contains(@class, 'js-row')]")
    for ele in table_rows :
        row_data = list(map(get_text, ele.find_elements_by_tag_name("td")))
        if (row_data[0]=='1'):
            row_data.extend([ele.get_attribute('data-nationality'), season, ele.get_attribute('class').split()[2]])
        else:
            row_data.extend([ele.get_attribute('data-nationality'), season, ele.get_attribute('class').split()[1]])
        df_wickets.loc[len(df_wickets)] = row_data
7

In [85]:
df_wickets.to_excel('data.xlsx')

In [88]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

fromaddr = 'srinidhi7058@gmail.com'
toaddr = ['srinidhi7058@gmail.com']

msg = MIMEMultipart()

msg['From'] = fromaddr
msg['To'] = ", ".join(toaddr)
msg['Subject'] = "Hello"

body = "Sample Mail"

msg.attach(MIMEText(body, 'plain'))

filename = "Data.xlsx"
attachment = open(r"C:\Users\srini\data.xlsx", 
"rb")

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "Lifeisgood@2207") #Type Password
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

(221, b'2.0.0 closing connection t5sm11289501pgb.58 - gsmtp')